# <div style="background:#FFFFAA">
<img src="logo.jpg", width=150, ALIGN="left", border=20>
<center>
<h1>Sample Starting Kit </h1>
 <br>This code was tested with <br>
Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) (https://anaconda.org/)<br>
<i> Adapted for Chalab by Isabelle Guyon from original code of Balázs Kégl</i> <br>
<a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science (CDS)</a>
</center>
<br>
<br>
<br>
<p>
ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS". The CDS, CHALEARN, AND/OR OTHER ORGANIZERS OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR ANY PARTICULAR PURPOSE, AND THE WARRANTY OF NON-INFRIGEMENT OF ANY THIRD PARTY'S INTELLECTUAL PROPERTY RIGHTS. IN NO EVENT SHALL AUTHORS AND ORGANIZERS BE LIABLE FOR ANY SPECIAL, 
INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF SOFTWARE, DOCUMENTS, MATERIALS, PUBLICATIONS, OR INFORMATION MADE AVAILABLE FOR THE CHALLENGE. 
</div>

<div style="background:#FFFFAA">
    <h2>Introduction </h2>
    <p>
In an era where computer graphics techniques for image generation are reaching stunning levels of quality, it becomes more and more challenging to detect fake from true, authentic images. However, this raises a lot of legal issues, mainly dealing with forgery. This project focuses mainly on the efficiency of Generative Adversarial Network (GAN) algorithms for producing art forgery and we will try to beat the state-of-the-art models that detect it. In fact, the Fine Arts Expert Institute in Geneva estimates that as much as 50 percent of artworks currently in circulation may be forgeries. Although the standard approach for image classification which is deep neural networks and especially Convolutional Neural Networks (CNNs) works very well nowadays, GAN performs strongly on generating fake images. By focusing on the art forgery aspect, we will try to bring a more specific perspective on the issue of image forgery.

</div>

### Requirements
The next cell will install all the required dependencies on your computer. You should consider replacing pip with pip3 if pip is related to python2.7 on your computer, or comment it if you already have the dependencies/are running in the docker of the challenge (runnable with the name mm886/codalab-legacy:latest if you know how to run a docker).

In [1]:
#!pip install --user -r requirements.txt

### Load the dataset
Because our dataset is very large (~16Go), so we have used numpy.memmap to load the data. We suggest strongly that you use the same method to load the data and choose a small batch_size (100~500) when you train your classifieur. If not, your computer will not respond.

#### Constants and libraries

In [2]:
%reload_ext autoreload
%matplotlib inline
%reload_ext autoreload
%autoreload 2
model_dir = 'sample_code_submission/'                       
result_dir = 'sample_result_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
import seaborn as sns; sns.set()
import warnings
import numpy as np
import pandas as pd
import tqdm
from matplotlib import pyplot as plt
import os
warnings.simplefilter(action='ignore', category=FutureWarning)

<div style="background:#FFFFAA">
    <h1> Step 1: Exploratory data analysis </h1>
<p>
We provide sample_data with the starting kit, but to prepare your submission, you must fetch the public_data from the challenge website and point to it.
    <br>
</div>

In [3]:
data_dir = 'public_data'
data_name = 'perso'
!ls $data_dir

'ls' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [4]:
f = open("public_data/perso_public.info","r")
for line in f : 
    print(line)

usage = 'Sample dataset perso data'

name = 'perso'

task = 'bi-class.classification'

target_type = 'Numerical'

feat_type = 'Numerical'

metric = 'bac_metric'

time_budget =  1200

feat_num =     200

target_num =     2

label_num =     2

train_num =   50000

valid_num =    9408

test_num =    18817

has_categorical =     0

has_missing =     0

is_sparse =     0



### 
The code below is to load the three datasets train, valid and test in the memory.

<div style="color:#FF7000;">
        <p>The following cell import the preprocessor</p>
</div>

In [5]:
from preprocessor import Preprocessor
prepTrain = Preprocessor()
prepTest = Preprocessor()
prepValid = Preprocessor()

In [6]:
def count_lines(file) : 
    count = -1
    for count,line in enumerate(open(file,'r')):
        pass
        count += 1
    return count
train_lines = count_lines('public_data/perso_train.solution')
valid_lines = count_lines('public_data/perso_valid.data')
test_lines = count_lines('public_data/perso_test.data')

<div style="color:#FF7000;">
        <p>The next 6 cells load the datasets and preprocess them</p>
</div>

In [7]:
train_data= np.memmap( 'public_data/train_data', dtype='float64', mode='w+',shape=(train_lines,12288))
reader = pd.read_table('public_data/perso_train.data', sep=' ', chunksize=1000,dtype='uint8',header=None)
i=0
for chunk in reader:
    chunk = prepTrain.standardizing(chunk)
    prepTrain.partial_fit(chunk)
    if(train_lines-i<1000) :
        train_data[i:] = chunk
        break
    else :
        train_data[i:i+1000]=chunk
        i+=1000
X_train = np.memmap( 'public_data/train_data', dtype='float64', mode='r',shape=(train_lines,12288))
train_solution= np.memmap( 'public_data/train_solution', dtype='uint8', mode='w+',shape=(train_lines,1))
reader = pd.read_table('public_data/perso_train.solution', chunksize=1000,dtype='uint8',header=None)
i=0
for chunk in reader:
    if(train_lines-i<1000) :
        train_solution[i:] = chunk
        break
    else : 
        train_solution[i:i+1000]=chunk
        i+=1000
y_train= np.memmap('public_data/train_solution', dtype='uint8', mode='r', shape=(train_lines,1))

In [8]:
X_train = prepTrain.transform(X_train)

Transforming 



In [9]:
valid_data= np.memmap( 'public_data/valid_data', dtype='float64', mode='w+',shape=(valid_lines,12288))
reader = pd.read_table('public_data/perso_valid.data', sep=' ', chunksize=1000,dtype='uint8',header=None)
i=0
for chunk in reader:
    chunk = prepValid.standardizing(chunk)
    prepValid.partial_fit(chunk)
    if(valid_lines-i < 1000) :
        valid_data[i:] = chunk
        break
    else :
        valid_data[i:i+1000]=chunk
        i+=1000
X_valid = np.memmap( 'public_data/valid_data', dtype='float64', mode='r',shape=(valid_lines,12288))
if(os.path.exists('public_data/perso_valid.solution')) :
    valid_solution= np.memmap( 'public_data/valid_solution', dtype='uint8', mode='w+',shape=(valid_lines,1))
    reader = pd.read_table('public_data/perso_valid.solution', chunksize=1000,dtype='uint8',header=None)
    i=0
    for chunk in reader:
        if(valid_lines-i<1000) : 
            valid_solution[i:] = chunk
            break
        else :
            valid_solution[i:i+1000]=chunk
            i+=1000
    y_valid= np.memmap('public_data/valid_solution', dtype='uint8', mode='r', shape=(valid_lines,1))

In [10]:
X_valid = prepValid.transform(X_valid)

Transforming 



In [11]:
test_data= np.memmap( 'public_data/test_data', dtype='float64', mode='w+',shape=(test_lines,12288))
reader = pd.read_table('public_data/perso_test.data', sep=' ', chunksize=1000,dtype='uint8',header=None)
i=0
for chunk in reader:
    chunk = prepTest.standardizing(chunk)
    prepTest.partial_fit(chunk)
    if(test_lines-i<1000) : 
        test_data[i:] = chunk
        break
    else :
        test_data[i:i+1000]=chunk
        i+=1000
X_test = np.memmap( 'public_data/test_data', dtype='float64', mode='r',shape=(test_lines,12288))
if(os.path.exists('public_data/perso_test.solution')) :
    test_solution= np.memmap( 'public_data/test_solution', dtype='uint8', mode='w+',shape=(test_lines,1))
    reader = pd.read_table('public_data/perso_test.solution', chunksize=1000,dtype='uint8',header=None)
    i=0
    for chunk in reader:
        if(test_lines-i<1000) : 
            test_solution[i:] = chunk
            break
        else :
            test_solution[i:i+1000]=chunk
            i+=1000
    y_test= np.memmap('public_data/test_solution', dtype='uint8', mode='r', shape=(test_lines,1))

In [12]:
X_test = prepTest.transform(X_test)

Transforming 



In [13]:
X_train[0]

array([ 7.10121755e+01, -4.78935761e+01,  1.45043970e+01,  1.56731624e+01,
       -8.98853417e+00,  9.48929764e+00, -6.92145875e+00,  9.62794615e+00,
       -2.20312050e+01, -1.00919320e+01,  2.01780369e-01, -8.31331098e-01,
       -6.03743182e+00, -2.44950539e+01,  1.82779930e+01, -4.36246217e+00,
       -1.55848181e+00,  1.53888815e+01,  8.25220377e+00,  8.35008873e+00,
       -1.62951679e+01,  5.54696889e+00, -2.09417200e+00, -4.14544222e+00,
        8.05808766e+00,  3.04500719e+00,  3.71857088e+00,  7.75000427e+00,
       -4.59259990e+00, -1.30142806e+00,  1.08080036e+01, -7.27083699e+00,
       -4.40413189e+00, -4.25062929e+00,  7.28127432e+00, -1.49271786e+00,
        2.25749343e+00, -2.15799771e+00, -1.00845809e+01, -2.09871484e+01,
       -8.54418274e+00,  4.48586639e-01, -2.79109919e-01, -2.37714543e+00,
        8.82517809e+00, -2.73154901e+00,  9.35936799e+00, -3.28827306e-01,
       -6.93559233e+00, -3.01059165e+00,  1.36422472e+01, -2.47840229e+00,
       -9.77295572e-01, -

<div style="background:#FFFFAA">
<h1>Step 2: Building a predictive model</h1>
</div>

<div style="background:#FFFFAA">
    <h2>Training a predictive model</h2>
    <p>
We provide an example of predictive model (for classification or regression) in the `sample_code_submission/` directory. It is a quite stupid model: it makes constant predictions. Replace it with your own model.
    </div>

In [14]:
from numpy.core.umath_tests import inner1d
from data_io import write
from model import model

C:\Users\mlans\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  """Entry point for launching an IPython kernel.
Using TensorFlow backend.


<div style="background:#FFFFAA">
an instance of the model (run the constructor) and attempt to reload a previously saved version from `sample_code_submission/`:
    
</div>

In [15]:
M = model()
trained_model_name = model_dir + data_name
# Uncomment the next line to re-load an already trained model
#M = M.load(trained_model_name)  

Instructions for updating:
Colocations handled automatically by placer.


<div style="background:#FFFFAA">
    Train the model (unless you reloaded a trained model) and make predictions. 
</div>

In [16]:
M.fit(X_train, y_train)

ValueError: Error when checking input: expected dense_1_input to have shape (800,) but got array with shape (400,)

In [ ]:
Y_hat_train = M.predict(X_train)
Y_hat_valid = M.predict(X_valid)
Y_hat_test = M.predict(X_test)

<div style="background:#FFFFAA">
    <b> Save the trained model </b> (will be ready to reload next time around) and save the prediction results. IMPORTANT: if you save the trained model, it will be bundled with your sample code submission. Therefore your model will NOT be retrained on the challenge platform. Remove the pickle from the submission if you want the model to be retrained on the platform.
</div>

In [ ]:
result_name = result_dir + data_name


from data_io import write
write(result_name + '_train.predict', Y_hat_train)
write(result_name + '_valid.predict', Y_hat_valid)
write(result_name + '_test.predict', Y_hat_test)
!ls $result_name*


<div style="background:#FFFFAA">
    <h2>Scoring the results</h2>
    <h3>Load the challenge metric</h3>
    <p>
<b>The metric chosen for your challenge</b> is identified in the "metric.txt" file found in the `scoring_function/` directory. The function "get_metric" searches first for a metric having that name in my_metric.py, then in libscores.py, then in sklearn.metric.
    </div>

#### Metric : AUC_metric 
ROC curves are typically used in binary classification to study the output of a classifier

In [ ]:
from libscores import get_metric
metric_name, scoring_function = get_metric()
print('Using scoring metric:', metric_name)

<div style="background:#FFFFAA">
    <h3> Training performance </h3>
    <p>
The participants normally posess target values (labels) only for training examples (except for the sample data). We compute with the `example` metric the training score, which should be zero for perfect predictions.
        </div>

You can add here other scores and result visualization.

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
n_classes=2
def fpr_tpr(solution, prediction):
    for i in range(n_classes):
        fpr, tpr, _ = metrics.roc_curve(solution, prediction)
        roc_auc = metrics.auc(fpr, tpr)
    return (fpr,tpr)

def p2c(prediction,threshold=0.5) : 
    c = []
    for ele in prediction : 
        if(ele>=0.5) : 
            c.append(1)
        else : 
            c.append(0)
    return np.array(c)
def plot_cm_matrix(solution,prediction,title) :
    prediction = p2c(prediction)
    cm = confusion_matrix(solution, prediction)
    df_cm = pd.DataFrame(cm, index = [i for i in "01"],columns = [i for i in "01"])
    plt.figure(figsize = (5,3))
    sn.heatmap(df_cm, annot=True)
    plt.title(title)
def plot_ROC(fpr,tpr,title) :
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',lw=lw)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.show()

fpr_train,tpr_train = fpr_tpr(y_train, Y_hat_train)
print('Training score for the', metric_name, 'metric = %5.4f' % scoring_function(y_train, Y_hat_train))
print('Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(y_train, y_train))
plot_cm_matrix(y_train,Y_hat_train,"Confusion matrix for train data") 
plot_ROC(fpr_train,tpr_train,"ROC curve for train data")

if(os.path.exists('public_data/perso_test.solution') and os.path.exists('public_data/perso_valid.solution')) :
    fpr_test,tpr_test = fpr_tpr(y_test, Y_hat_test)
    fpr_valid,tpr_valid = fpr_tpr(y_valid, Y_hat_valid)
    print('Test score for the', metric_name, 'metric = %5.4f' % scoring_function(y_test, Y_hat_test))
    print('Valid score for the', metric_name, 'metric = %5.4f' % scoring_function(y_valid, Y_hat_valid))
    plot_cm_matrix(y_test,Y_hat_test,"Confusion matrix for test data") 
    plot_ROC(fpr_test,tpr_test,"ROC curve for test data")
    plot_cm_matrix(y_valid,Y_hat_valid,"Confusion matrix for valid data") 
    plot_ROC(fpr_valid,tpr_valid,"ROC curve for valid data")

<div style="background:#FFFFAA">
    <h3>Cross-validation performance</h3>
    <p>
The participants do not have access to the labels Y_valid and Y_test to self-assess their validation and test performances. But training performance is not a good prediction of validation or test performance. Using cross-validation, the training data is split into multiple training/test folds, which allows participants to self-assess their model during development. The average CV result and 95% confidence interval is displayed.
   </div>

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
scores = cross_val_score(M, X_train, y_train, cv=2, scoring=make_scorer(scoring_function))
print('\nCV score (95 perc. CI): %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

<div style="background:#FFFFAA">
<h1> Step 3: Making a submission </h1> 



In [ ]:
scoring_output_dir = '../scoring_output_dir'
!python $score_dir/score.py $data_dir $result_dir $scoring_output_dir

<div style="background:#FFFFAA">
    <h1> Preparing the submission </h1>

Zip the contents of `sample_code_submission/` (without the directory), or download the challenge public_data and run the command in the previous cell, after replacing sample_data by public_data.
Then zip the contents of `sample_result_submission/` (without the directory).
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>.

In [ ]:
import datetime 
from data_io import zipdir
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
sample_code_submission = '../sample_code_submission_' + the_date + '.zip'
sample_result_submission = '../sample_result_submission_' + the_date + '.zip'
zipdir(sample_code_submission, model_dir)
zipdir(sample_result_submission, result_dir)
print("Submit one of these files:\n" + sample_code_submission + "\n" + sample_result_submission)